# CUSTOMER CHURN PREDICTION USING MACHINE LEARNING
# Bachelor's Thesis - E-Commerce Dataset

## 1. Introduction

This notebook implements customer churn prediction for an e-commerce dataset.

### Methodology:
1. Data Loading & Exploration
2. Preprocessing & Feature Engineering
3. Model Training with Cross-Validation
4. Hyperparameter Tuning
5. Statistical Significance Testing
6. Feature Importance Analysis
7. Results & Conclusions

## 2. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score,
    RandomizedSearchCV
)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from scipy.stats import wilcoxon
from itertools import combinations
import warnings

warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('Libraries imported successfully')

Libraries imported successfully


## 3. Data Loading

In [2]:
# Load E-Commerce dataset (try Excel first, fall back to CSV)
try:
    df = pd.read_excel('/Users/nukesaba/PyCharmMiscProject/ECommerceDataset.xlsx')
except:
    df = pd.read_csv('/Users/nukesaba/PyCharmMiscProject/ECommerceDataset.csv')

target = 'Churn'

# Drop CustomerID as it's just an identifier
if 'CustomerID' in df.columns:
    df = df.drop(['CustomerID'], axis=1)

print('Dataset Shape:', df.shape)
print('\nFirst 5 rows:')
print(df.head())
print('\nTarget Distribution:')
print(df[target].value_counts())
print(f'\nImbalance Ratio: {df[target].value_counts()[0] / df[target].value_counts()[1]:.2f}:1')

Dataset Shape: (5630, 19)

First 5 rows:
   Churn  Tenure PreferredLoginDevice  CityTier  WarehouseToHome  \
0      1     4.0         Mobile Phone         3              6.0   
1      1     NaN                Phone         1              8.0   
2      1     NaN                Phone         1             30.0   
3      1     0.0                Phone         3             15.0   
4      1     0.0                Phone         1             12.0   

  PreferredPaymentMode  Gender  HourSpendOnApp  NumberOfDeviceRegistered  \
0           Debit Card  Female             3.0                         3   
1                  UPI    Male             3.0                         4   
2           Debit Card    Male             2.0                         4   
3           Debit Card    Male             2.0                         4   
4                   CC    Male             NaN                         3   

     PreferedOrderCat  SatisfactionScore MaritalStatus  NumberOfAddress  \
0  Laptop & Access

## 4. Data Preprocessing

In [3]:
print('Missing Values:')
print(df.isnull().sum())

print('\nData types:')
print(df.dtypes)

Missing Values:
Churn                            0
Tenure                         264
PreferredLoginDevice             0
CityTier                         0
WarehouseToHome                251
PreferredPaymentMode             0
Gender                           0
HourSpendOnApp                 255
NumberOfDeviceRegistered         0
PreferedOrderCat                 0
SatisfactionScore                0
MaritalStatus                    0
NumberOfAddress                  0
Complain                         0
OrderAmountHikeFromlastYear    265
CouponUsed                     256
OrderCount                     258
DaySinceLastOrder              307
CashbackAmount                   0
dtype: int64

Data types:
Churn                            int64
Tenure                         float64
PreferredLoginDevice            object
CityTier                         int64
WarehouseToHome                float64
PreferredPaymentMode            object
Gender                          object
HourSpendOnApp      

## 5. Train-Test Split & Feature Engineering

In [4]:
# CRITICAL: Split FIRST to prevent data leakage
X = df.drop(target, axis=1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')
print(f'\nClass distribution in training set:')
print(y_train.value_counts())
print(f'Imbalance ratio: {y_train.value_counts()[0]/y_train.value_counts()[1]:.2f}:1')

Training set: 4504 samples
Test set: 1126 samples

Class distribution in training set:
Churn
0    3746
1     758
Name: count, dtype: int64
Imbalance ratio: 4.94:1


In [5]:
# Handle missing values - compute statistics on TRAINING data only
X_train = X_train.copy()
X_test = X_test.copy()

# Fill numeric columns with median (training median)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if X_train[col].isnull().any():
        train_median = X_train[col].median()
        X_train[col] = X_train[col].fillna(train_median)
        X_test[col] = X_test[col].fillna(train_median)

# Fill categorical columns with mode (training mode)
categorical_cols = X_train.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if X_train[col].isnull().any():
        train_mode = X_train[col].mode()[0]
        X_train[col] = X_train[col].fillna(train_mode)
        X_test[col] = X_test[col].fillna(train_mode)

print('✓ Missing values handled using training set statistics')

✓ Missing values handled using training set statistics


In [6]:
# Encode categorical features - fit on TRAINING data only
le_gender = LabelEncoder()
le_marital = LabelEncoder()

X_train['Gender'] = le_gender.fit_transform(X_train['Gender'])
X_test['Gender'] = le_gender.transform(X_test['Gender'])

X_train['MaritalStatus'] = le_marital.fit_transform(X_train['MaritalStatus'])
X_test['MaritalStatus'] = le_marital.transform(X_test['MaritalStatus'])

# One-hot encode multi-category features
categorical_cols = ['PreferredLoginDevice', 'PreferredPaymentMode', 'PreferedOrderCat']
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Ensure test set has same columns as train set
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols:
    X_test[col] = 0

extra_cols = set(X_test.columns) - set(X_train.columns)
for col in extra_cols:
    X_test = X_test.drop(columns=[col])

X_test = X_test[X_train.columns]

print('After encoding:')
print(X_train.head())
print(f'\nFeature count: {X_train.shape[1]}')

After encoding:
      Tenure  CityTier  WarehouseToHome  Gender  HourSpendOnApp  \
1787     9.0         3             16.0       1             2.0   
2147     6.0         3             13.0       0             1.0   
1717     8.0         1             15.0       1             3.0   
2292    15.0         3             11.0       1             3.0   
5578    12.0         1             13.0       1             4.0   

      NumberOfDeviceRegistered  SatisfactionScore  MaritalStatus  \
1787                         3                  1              2   
2147                         3                  4              1   
1717                         4                  4              2   
2292                         3                  4              2   
5578                         5                  3              1   

      NumberOfAddress  Complain  ...  PreferredPaymentMode_Cash on Delivery  \
1787                2         0  ...                                  False   
2147          

In [7]:
# Feature Engineering - parameters computed on TRAINING data only

# Cashback efficiency
X_train['CashbackEfficiency'] = X_train['CashbackAmount'] / (X_train['OrderAmountHikeFromlastYear'] + 1e-6)
X_test['CashbackEfficiency'] = X_test['CashbackAmount'] / (X_test['OrderAmountHikeFromlastYear'] + 1e-6)

# Order frequency
X_train['OrderFrequency'] = X_train['OrderCount'] / (X_train['Tenure'] + 1)
X_test['OrderFrequency'] = X_test['OrderCount'] / (X_test['Tenure'] + 1)

# App usage intensity
X_train['AppUsageIntensity'] = X_train['HourSpendOnApp'] / (X_train['Tenure'] + 1)
X_test['AppUsageIntensity'] = X_test['HourSpendOnApp'] / (X_test['Tenure'] + 1)

# High value customer indicator (thresholds from TRAINING data)
order_threshold = X_train['OrderCount'].median()
cashback_threshold = X_train['CashbackAmount'].median()

X_train['IsHighValue'] = (
    (X_train['OrderCount'] > order_threshold) |
    (X_train['CashbackAmount'] > cashback_threshold)
).astype(int)

X_test['IsHighValue'] = (
    (X_test['OrderCount'] > order_threshold) |
    (X_test['CashbackAmount'] > cashback_threshold)
).astype(int)

# Tenure group binning (quantiles from TRAINING data)
q33 = X_train['Tenure'].quantile(0.33)
q66 = X_train['Tenure'].quantile(0.66)
tenure_bins = [-np.inf, q33, q66, np.inf]

X_train['TenureGroup'] = pd.cut(X_train['Tenure'], bins=tenure_bins, labels=[0, 1, 2]).astype(int)
X_test['TenureGroup'] = pd.cut(X_test['Tenure'], bins=tenure_bins, labels=[0, 1, 2]).astype(int)

print(f'Final feature count: {X_train.shape[1]}')

Final feature count: 33


In [8]:
# Scaling - fit on TRAINING data only
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print(f'Scaled {len(numeric_cols)} columns')

Scaled 20 columns


## 6. Model Training with Cross-Validation

SMOTE is applied INSIDE each CV fold using ImbPipeline to prevent data leakage.

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

base_models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'k-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Support Vector Machine': SVC(random_state=RANDOM_STATE),
    'XGBoost': xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'),
    'LightGBM': lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1),
    'CatBoost': CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
}

cv_results = {}

print('=' * 60)
print('CROSS-VALIDATION (SMOTE inside each fold)')
print('=' * 60)

for name, model in base_models.items():
    pipeline = ImbPipeline([
        ('smote', SMOTE(random_state=RANDOM_STATE)),
        ('classifier', model)
    ])

    scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='f1', n_jobs=-1)

    cv_results[name] = {
        'scores': scores,
        'mean': scores.mean(),
        'std': scores.std()
    }

    print(f'{name:25s}: F1 = {scores.mean():.4f} (+/-{scores.std():.4f})')

CROSS-VALIDATION (SMOTE inside each fold)
Logistic Regression      : F1 = 0.6235 (+/-0.0240)
Decision Tree            : F1 = 0.7401 (+/-0.0292)
k-Nearest Neighbors      : F1 = 0.7274 (+/-0.0114)
Naive Bayes              : F1 = 0.4975 (+/-0.0214)
Support Vector Machine   : F1 = 0.7557 (+/-0.0300)
XGBoost                  : F1 = 0.8660 (+/-0.0153)
LightGBM                 : F1 = 0.8354 (+/-0.0240)
CatBoost                 : F1 = 0.8368 (+/-0.0179)


In [10]:
cv_summary = pd.DataFrame({
    'Model': cv_results.keys(),
    'Mean F1': [cv_results[m]['mean'] for m in cv_results],
    'Std': [cv_results[m]['std'] for m in cv_results]
}).sort_values('Mean F1', ascending=False)

print('\n' + '=' * 60)
print('CV SUMMARY (Sorted by F1-Score)')
print('=' * 60)
print(cv_summary.to_string(index=False))


CV SUMMARY (Sorted by F1-Score)
                 Model  Mean F1      Std
               XGBoost 0.866036 0.015338
              CatBoost 0.836780 0.017920
              LightGBM 0.835389 0.024042
Support Vector Machine 0.755690 0.029973
         Decision Tree 0.740086 0.029173
   k-Nearest Neighbors 0.727351 0.011397
   Logistic Regression 0.623530 0.023986
           Naive Bayes 0.497516 0.021370


## 7. Hyperparameter Tuning

Tuning top 3 models. SMOTE is applied inside CV folds via ImbPipeline.

In [11]:
top_3_models = cv_summary.head(3)['Model'].tolist()
print(f'Top 3 models to tune: {top_3_models}')

Top 3 models to tune: ['XGBoost', 'CatBoost', 'LightGBM']


In [12]:
param_grids = {
    'LightGBM': {
        'classifier__num_leaves': [31, 50, 70],
        'classifier__learning_rate': [0.01, 0.05, 0.1],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__min_child_samples': [20, 30, 40]
    },
    'CatBoost': {
        'classifier__depth': [4, 6, 8],
        'classifier__learning_rate': [0.01, 0.05, 0.1],
        'classifier__iterations': [100, 200, 300],
        'classifier__l2_leaf_reg': [1, 3, 5, 7]
    },
    'XGBoost': {
        'classifier__max_depth': [3, 5, 7],
        'classifier__learning_rate': [0.01, 0.1, 0.2],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__subsample': [0.8, 0.9, 1.0],
        'classifier__colsample_bytree': [0.8, 0.9, 1.0]
    }
}

model_instances = {
    'LightGBM': lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1),
    'CatBoost': CatBoostClassifier(random_state=RANDOM_STATE, verbose=0),
    'XGBoost': xgb.XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss')
}

In [13]:
def tune_model(model_name, model, param_grid, X_train, y_train, n_iter=50):
    """Hyperparameter tuning with SMOTE inside CV folds."""
    print(f"\n{'=' * 60}")
    print(f'TUNING: {model_name}')
    print(f"{'=' * 60}")

    pipeline = ImbPipeline([
        ('smote', SMOTE(random_state=RANDOM_STATE)),
        ('classifier', model)
    ])

    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=skf,
        scoring='f1',
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1
    )

    search.fit(X_train, y_train)

    print(f'\nBest CV F1-Score: {search.best_score_:.4f}')
    print('Best Parameters:')
    for param, value in search.best_params_.items():
        print(f'  {param}: {value}')

    return search.best_estimator_, search.best_params_, search.best_score_

In [14]:
tuned_models = {}
tuning_results = {}

for model_name in top_3_models:
    if model_name in param_grids:
        best_pipeline, best_params, best_score = tune_model(
            model_name,
            model_instances[model_name],
            param_grids[model_name],
            X_train, y_train
        )
        tuned_models[f'{model_name} (Tuned)'] = best_pipeline
        tuning_results[model_name] = {
            'best_params': best_params,
            'cv_score': best_score
        }


TUNING: XGBoost
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Best CV F1-Score: 0.8685
Best Parameters:
  classifier__subsample: 1.0
  classifier__n_estimators: 300
  classifier__max_depth: 7
  classifier__learning_rate: 0.2
  classifier__colsample_bytree: 0.9

TUNING: CatBoost
Fitting 5 folds for each of 50 candidates, totalling 250 fits

Best CV F1-Score: 0.8746
Best Parameters:
  classifier__num_leaves: 70
  classifier__n_estimators: 200
  classifier__min_child_samples: 30
  classifier__learning_rate: 0.1


## 8. Model Evaluation on Test Set

In [15]:
def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate model and return metrics."""
    y_pred = model.predict(X_test)

    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }

    return metrics, y_pred

In [16]:
print('=' * 60)
print('TEST SET EVALUATION')
print('=' * 60)

evaluation_results = []

for name, model in tuned_models.items():
    metrics, y_pred = evaluate_model(model, X_test, y_test, name)
    evaluation_results.append(metrics)

    print(f'\n{name}')
    print('-' * 40)
    print(f"Accuracy:  {metrics['Accuracy']:.4f}")
    print(f"Precision: {metrics['Precision']:.4f}")
    print(f"Recall:    {metrics['Recall']:.4f}")
    print(f"F1-Score:  {metrics['F1-Score']:.4f}")
    print('\nClassification Report:')
    print(classification_report(y_test, y_pred))
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))

TEST SET EVALUATION

XGBoost (Tuned)
----------------------------------------
Accuracy:  0.9938
Precision: 0.9893
Recall:    0.9737
F1-Score:  0.9814

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       936
           1       0.99      0.97      0.98       190

    accuracy                           0.99      1126
   macro avg       0.99      0.99      0.99      1126
weighted avg       0.99      0.99      0.99      1126

Confusion Matrix:
[[934   2]
 [  5 185]]

CatBoost (Tuned)
----------------------------------------
Accuracy:  0.9911
Precision: 0.9839
Recall:    0.9632
F1-Score:  0.9734

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       936
           1       0.98      0.96      0.97       190

    accuracy                           0.99      1126
   macro avg       0.99      0.98      0.98      1126
weighted avg       0.99      0.

In [17]:
results_df = pd.DataFrame(evaluation_results).sort_values('F1-Score', ascending=False)

print('\n' + '=' * 60)
print('RESULTS SUMMARY')
print('=' * 60)
print(results_df.to_string(index=False))

best_model_name = results_df.iloc[0]['Model']
best_f1 = results_df.iloc[0]['F1-Score']
print(f'\nBest Model: {best_model_name} with F1-Score = {best_f1:.4f}')


RESULTS SUMMARY
           Model  Accuracy  Precision   Recall  F1-Score
 XGBoost (Tuned)  0.993783   0.989305 0.973684  0.981432
CatBoost (Tuned)  0.991119   0.983871 0.963158  0.973404
LightGBM (Tuned)  0.990231   0.989071 0.952632  0.970509

Best Model: XGBoost (Tuned) with F1-Score = 0.9814


## 9. Statistical Significance Testing

Wilcoxon signed-rank test to compare models.

In [ ]:
print('Collecting CV scores for statistical testing...')

cv_scores_dict = {}
for name, model in tuned_models.items():
    scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='f1', n_jobs=-1)
    cv_scores_dict[name] = scores
    print(f'{name:25s}: {scores.mean():.4f} (+/-{scores.std():.4f})')

XGBoost (Tuned)          : 0.8685 (+/-0.0131)


In [ ]:
def wilcoxon_test(scores_a, scores_b, name_a, name_b):
    """Wilcoxon signed-rank test between two models."""
    statistic, p_value = wilcoxon(scores_a, scores_b)

    diff = scores_a - scores_b
    cohens_d = np.mean(diff) / np.std(diff, ddof=1) if np.std(diff, ddof=1) > 0 else 0

    return {
        'Model A': name_a,
        'Model B': name_b,
        'Mean A': np.mean(scores_a),
        'Mean B': np.mean(scores_b),
        'p-value': p_value,
        "Cohen's d": cohens_d,
        'Significant': p_value < 0.05
    }

In [ ]:
print('\n' + '=' * 60)
print('STATISTICAL SIGNIFICANCE TESTING')
print('=' * 60)

model_names = list(cv_scores_dict.keys())
comparisons = list(combinations(model_names, 2))

alpha = 0.05
corrected_alpha = alpha / len(comparisons) if len(comparisons) > 0 else alpha
print(f'Comparisons: {len(comparisons)}')
print(f'Bonferroni-corrected alpha: {corrected_alpha:.6f}')

significance_results = []
for name_a, name_b in comparisons:
    result = wilcoxon_test(
        cv_scores_dict[name_a],
        cv_scores_dict[name_b],
        name_a, name_b
    )
    result['Significant (Bonferroni)'] = result['p-value'] < corrected_alpha
    significance_results.append(result)

if len(significance_results) > 0:
    significance_df = pd.DataFrame(significance_results)
    print('\n' + significance_df.to_string(index=False))

    n_significant = significance_df['Significant'].sum()
    n_bonferroni = significance_df['Significant (Bonferroni)'].sum()

    print(f'\nSignificant at alpha=0.05: {n_significant}/{len(comparisons)}')
    print(f'Significant with Bonferroni: {n_bonferroni}/{len(comparisons)}')
else:
    print('\nNot enough models for pairwise comparisons')

## 10. Feature Importance Analysis

In [ ]:
def get_feature_importance(model, feature_names, model_name):
    """Extract feature importance from model."""
    if hasattr(model, 'named_steps'):
        classifier = model.named_steps['classifier']
    else:
        classifier = model

    if hasattr(classifier, 'feature_importances_'):
        importance = classifier.feature_importances_
        return pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance
        }).sort_values('Importance', ascending=False)
    else:
        print(f'{model_name} does not have feature_importances_')
        return None

In [ ]:
feature_names = X_train.columns.tolist()

print('=' * 60)
print('FEATURE IMPORTANCE')
print('=' * 60)

for name, model in tuned_models.items():
    importance_df = get_feature_importance(model, feature_names, name)
    if importance_df is not None:
        print(f'\n{name} - Top 10 Features:')
        print(importance_df.head(10).to_string(index=False))

In [ ]:
best_model = tuned_models[best_model_name]
importance_df = get_feature_importance(best_model, feature_names, best_model_name)

if importance_df is not None:
    plt.figure(figsize=(10, 8))
    top_15 = importance_df.head(15)

    plt.barh(range(len(top_15)), top_15['Importance'], color='steelblue')
    plt.yticks(range(len(top_15)), top_15['Feature'])
    plt.xlabel('Importance')
    plt.title(f'Top 15 Features - {best_model_name}', fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('Ecom/feature_importance_ecommerce.png', dpi=300, bbox_inches='tight')
    plt.show()

## 11. Confusion Matrix Visualization

In [ ]:
fig, axes = plt.subplots(1, len(tuned_models), figsize=(5 * len(tuned_models), 4))
if len(tuned_models) == 1:
    axes = [axes]

for idx, (name, model) in enumerate(tuned_models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                annot_kws={'fontsize': 14, 'fontweight': 'bold'})
    axes[idx].set_title(f'{name}', fontweight='bold')
    axes[idx].set_ylabel('Actual')
    axes[idx].set_xlabel('Predicted')

plt.tight_layout()
plt.savefig('Ecom/confusion_matrices_ecommerce.png', dpi=300, bbox_inches='tight')
plt.show()

## 12. Conclusions

In [ ]:
print('=' * 70)
print('FINAL SUMMARY')
print('=' * 70)

print('\n1. BEST MODEL:')
print(f'   {best_model_name}')
print(f'   F1-Score: {best_f1:.4f}')
print(f"   Accuracy: {results_df.iloc[0]['Accuracy']:.4f}")

print('\n2. ALL MODELS:')
print(results_df.to_string(index=False))

print('\n3. STATISTICAL SIGNIFICANCE:')
if len(comparisons) > 0 and len(significance_results) > 0:
    if n_bonferroni > 0:
        print(f'   {n_bonferroni} significant difference(s) found')
    else:
        print('   No significant differences - models perform comparably')
else:
    print('   Not enough models for comparison')

print('\n4. TOP 5 FEATURES:')
if importance_df is not None:
    for idx, row in importance_df.head(5).iterrows():
        print(f"   {row['Feature']}: {row['Importance']:.4f}")

print('\n' + '=' * 70)